In [1]:
import CropModels
from CropDataset import MyDataSet,normalize_torch,normalize_05,normalize_dataset,preprocess,preprocess_hflip,preprocess_with_augmentation
import pandas as pd
from torch.utils.data import DataLoader
import torch.nn as nn
from tensorboardX import SummaryWriter
import datetime
import os
import torch
from torch.autograd import Variable
from utils import RunningMean
import utils
from PIL import Image
import numpy as np
import random

In [2]:
NB_CLASS=59
SEED=888

In [3]:
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
torch.backends.cudnn.benchmark = True

In [4]:
BATCH_SIZE=32
IMAGE_SIZE=224    # 不同模型修改不同的Size
IMAGE_TRAIN_PRE='../data/AgriculturalDisease_trainingset/images/'
ANNOTATION_TRAIN='../data/AgriculturalDisease_trainingset/AgriculturalDisease_train_annotations_deleteNoise.json' #是否需要剔除两类异常类
IMAGE_VAL_PRE='../data/AgriculturalDisease_validationset/images/'
ANNOTATION_VAL='../data/AgriculturalDisease_validationset/AgriculturalDisease_validation_annotations_deleteNoise.json' #是否需要剔除两类异常类
date=str(datetime.date.today())
with open(ANNOTATION_TRAIN) as datafile1:
    trainDataFram=pd.read_json(datafile1,orient='records')
with open(ANNOTATION_VAL) as datafile2: #first check if it's a valid json file or not
    validateDataFram =pd.read_json(datafile2,orient='records')    
def getmodel():
    print('[+] loading model... ', end='', flush=True)
    model=CropModels.resnet152_finetune(NB_CLASS)
    model.cuda()
    print('Done')
    return model


In [5]:
def train(epochNum):
    writer=SummaryWriter('log/'+date+'/ResNet/') # 创建 /log/日期/InceptionResnet的组织形式  不同模型需要修改不同名称
    train_dataset=MyDataSet(json_Description=ANNOTATION_TRAIN,transform=preprocess_with_augmentation(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_TRAIN_PRE)
    val_dataset=MyDataSet(json_Description=ANNOTATION_VAL,transform=preprocess(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_VAL_PRE)
    train_dataLoader=DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,num_workers=16,shuffle=True)
    val_dataLoader=DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,num_workers=1,shuffle=False)
    model=getmodel()
    weight=torch.Tensor([1,3,3,3,3,4,2,3,3,3,3,3,3,3,3,3,2,3,3,3,2,3,4,2,3,1,1,3,2,2,1,3,3,1,3,2,3,3,3,3,2,1,3,2,3,3,3,1,3,3,4,4,3,2,2,3,1,1,3]).cuda()
    criterion=nn.CrossEntropyLoss(weight=weight).cuda()
#     lx, px = utils.predict(model,val_dataLoader)
#     min_loss = criterion(Variable(px), Variable(lx)).item()
    min_loss=4.1
    print('min_loss is :%f'%(min_loss))
    min_acc=0.80
    patience=0
    lr=0.0
    momentum=0.0
    for epoch in range(epochNum):
        print('Epoch {}/{}'.format(epoch, epochNum - 1))
        print('-' * 10)
        if epoch==3:
            lr=1e-4
            momentum=0.9
            print('set lr=:%f,momentum=%f'%(lr,momentum))
        if patience==2:
            patience=0
            model.load_state_dict(torch.load('../model/ResNet/'+date+'_loss_best.pth')['state_dict'])
            lr=lr/10
            print('loss has increased lr divide 10 lr now is :%f'%(lr))
        if epoch==0 or epoch==1 or epoch==2: #第一轮首先训练全连接层
            lr=1e-3
#             optimizer=torch.optim.SGD(params=model.fresh_params(),lr=lr,momentum=0.9)
            optimizer = torch.optim.Adam(model.fresh_params(),lr = lr,amsgrad=True,weight_decay=1e-4)
        else:
            optimizer = torch.optim.Adam(model.parameters(),lr = lr,amsgrad=True,weight_decay=1e-4)
#             optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
        running_loss = RunningMean()
        running_corrects = RunningMean()
        for batch_idx, (inputs, labels) in enumerate(train_dataLoader):
            model.train(True)
            n_batchsize=inputs.size(0)
            inputs = Variable(inputs).cuda()
            labels = Variable(labels).cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            if isinstance(outputs,tuple):
                loss=sum((criterion(o,labels)) for o in outputs)
            else:
                loss = criterion(outputs, labels)
            running_loss.update(loss.item(),1)
            running_corrects.update(torch.sum(preds == labels.data).data,n_batchsize)
            loss.backward()
            optimizer.step()
            if batch_idx%30==29:
                print('[epoch:%d,batch:%d]:acc: %f,loss:%f'%(epoch,batch_idx,running_corrects.value,running_loss.value))
                if batch_idx%300==299: 
                    niter = epoch * len(train_dataset)/BATCH_SIZE + batch_idx
                    writer.add_scalar('Train/Acc',running_corrects.value,niter)
                    writer.add_scalar('Train/Loss',running_loss.value,niter)
                    lx,px=utils.predict(model,val_dataLoader)
                    log_loss = criterion(Variable(px), Variable(lx))
                    log_loss = log_loss.item()
                    _, preds = torch.max(px, dim=1)
                    accuracy = torch.mean((preds == lx).float())
                    writer.add_scalar('Val/Acc',accuracy,niter)
                    writer.add_scalar('Val/Loss',log_loss,niter)
                    print('[epoch:%d,batch:%d]: val_loss:%f,val_acc:%f,val_total:%d'%(epoch,batch_idx,log_loss,accuracy,len(val_dataset)))
        print('[epoch:%d] :acc: %f,loss:%f,lr:%f,patience:%d'%(epoch,running_corrects.value,running_loss.value,lr,patience))       
        lx,px=utils.predict(model,val_dataLoader)
        log_loss = criterion(Variable(px), Variable(lx))
        log_loss = log_loss.item()
        _, preds = torch.max(px, dim=1)
        accuracy = torch.mean((preds == lx).float())
        writer.add_scalar('Val/Acc',accuracy,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        writer.add_scalar('Val/Loss',log_loss,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        print('[epoch:%d]: val_loss:%f,val_acc:%f,'%(epoch,log_loss,accuracy))
        if  log_loss < min_loss:
            utils.snapshot('../model/', 'ResNet', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy })          
            patience = 0
            min_loss=log_loss
            print('save new model loss,now loss is ',min_loss)
        else:
            patience += 1
        if accuracy>min_acc:
            utils.snapshot('../model/', 'ResNet', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy },key='acc') 
            min_acc=accuracy
            print('save new model acc,now acc is ',min_acc)

In [ ]:
#train(60)

[+] loading model... Done
min_loss is :4.100000
Epoch 0/59
----------
[epoch:0,batch:29]:acc: 0.182292,loss:3.888799
[epoch:0,batch:59]:acc: 0.280208,loss:3.396592
[epoch:0,batch:89]:acc: 0.347569,loss:3.006880
[epoch:0,batch:119]:acc: 0.396354,loss:2.756406
[epoch:0,batch:149]:acc: 0.432292,loss:2.553408
[epoch:0,batch:179]:acc: 0.461458,loss:2.415579
[epoch:0,batch:209]:acc: 0.480357,loss:2.301396
[epoch:0,batch:239]:acc: 0.499349,loss:2.202862
[epoch:0,batch:269]:acc: 0.515625,loss:2.112860
[epoch:0,batch:299]:acc: 0.524896,loss:2.043763
[epoch:0,batch:299]: val_loss:1.086678,val_acc:0.705442,val_total:4539
[epoch:0,batch:329]:acc: 0.537405,loss:1.969479
[epoch:0,batch:359]:acc: 0.544792,loss:1.912062
[epoch:0,batch:389]:acc: 0.552885,loss:1.863799
[epoch:0,batch:419]:acc: 0.560491,loss:1.813773
[epoch:0,batch:449]:acc: 0.567431,loss:1.768813
[epoch:0,batch:479]:acc: 0.572461,loss:1.732220
[epoch:0,batch:509]:acc: 0.578370,loss:1.695121
[epoch:0,batch:539]:acc: 0.584838,loss:1.65845

[epoch:4,batch:119]:acc: 0.811198,loss:0.514957
[epoch:4,batch:149]:acc: 0.808125,loss:0.511416
[epoch:4,batch:179]:acc: 0.807465,loss:0.515490
[epoch:4,batch:209]:acc: 0.808185,loss:0.521796
[epoch:4,batch:239]:acc: 0.808464,loss:0.524954
[epoch:4,batch:269]:acc: 0.809722,loss:0.518436
[epoch:4,batch:299]:acc: 0.810312,loss:0.517052
[epoch:4,batch:299]: val_loss:0.552869,val_acc:0.806565,val_total:4539
[epoch:4,batch:329]:acc: 0.808712,loss:0.518569
[epoch:4,batch:359]:acc: 0.807986,loss:0.518524
[epoch:4,batch:389]:acc: 0.807532,loss:0.515180
[epoch:4,batch:419]:acc: 0.809152,loss:0.510038
[epoch:4,batch:449]:acc: 0.809236,loss:0.507527
[epoch:4,batch:479]:acc: 0.809245,loss:0.508282
[epoch:4,batch:509]:acc: 0.808517,loss:0.509624
[epoch:4,batch:539]:acc: 0.808623,loss:0.507373
[epoch:4,batch:569]:acc: 0.808498,loss:0.507132
[epoch:4,batch:599]:acc: 0.808646,loss:0.506530
[epoch:4,batch:599]: val_loss:0.497001,val_acc:0.799295,val_total:4539
[epoch:4,batch:629]:acc: 0.808581,loss:0.5

[epoch:8,batch:299]: val_loss:0.384681,val_acc:0.857458,val_total:4539
[epoch:8,batch:329]:acc: 0.874621,loss:0.287535
[epoch:8,batch:359]:acc: 0.877517,loss:0.284146
[epoch:8,batch:389]:acc: 0.876683,loss:0.286941
[epoch:8,batch:419]:acc: 0.877753,loss:0.287166
[epoch:8,batch:449]:acc: 0.877569,loss:0.287028
[epoch:8,batch:479]:acc: 0.878060,loss:0.286750
[epoch:8,batch:509]:acc: 0.878738,loss:0.285839
[epoch:8,batch:539]:acc: 0.879514,loss:0.285790
[epoch:8,batch:569]:acc: 0.879112,loss:0.286083
[epoch:8,batch:599]:acc: 0.879844,loss:0.285001
[epoch:8,batch:599]: val_loss:0.401345,val_acc:0.849747,val_total:4539
[epoch:8,batch:629]:acc: 0.878869,loss:0.287472
[epoch:8,batch:659]:acc: 0.877936,loss:0.289071
[epoch:8,batch:689]:acc: 0.877672,loss:0.289436
[epoch:8,batch:719]:acc: 0.877778,loss:0.288239
[epoch:8,batch:749]:acc: 0.877125,loss:0.287742
[epoch:8,batch:779]:acc: 0.876963,loss:0.288942
[epoch:8,batch:809]:acc: 0.876427,loss:0.290495
[epoch:8,batch:839]:acc: 0.876600,loss:0.2

[epoch:12,batch:359]:acc: 0.880295,loss:0.295763
[epoch:12,batch:389]:acc: 0.879968,loss:0.296310
[epoch:12,batch:419]:acc: 0.879539,loss:0.296985
[epoch:12,batch:449]:acc: 0.877847,loss:0.299536
[epoch:12,batch:479]:acc: 0.876953,loss:0.302383
[epoch:12,batch:509]:acc: 0.876961,loss:0.301628
[epoch:12,batch:539]:acc: 0.876505,loss:0.301968
[epoch:12,batch:569]:acc: 0.875439,loss:0.304161
[epoch:12,batch:599]:acc: 0.876042,loss:0.303205
[epoch:12,batch:599]: val_loss:0.375603,val_acc:0.859220,val_total:4539
[epoch:12,batch:629]:acc: 0.876538,loss:0.302276
[epoch:12,batch:659]:acc: 0.876610,loss:0.302057
[epoch:12,batch:689]:acc: 0.877174,loss:0.302394
[epoch:12,batch:719]:acc: 0.876953,loss:0.304374
[epoch:12,batch:749]:acc: 0.876833,loss:0.304188
[epoch:12,batch:779]:acc: 0.876442,loss:0.304025
[epoch:12,batch:809]:acc: 0.876427,loss:0.303933
[epoch:12,batch:839]:acc: 0.876600,loss:0.304259
[epoch:12,batch:869]:acc: 0.877011,loss:0.303216
[epoch:12,batch:899]:acc: 0.876806,loss:0.3025

In [ ]:
def reuseTrain(path,epochNum):
    writer=SummaryWriter('log/'+date+'/ResNet/') # 创建 /log/日期/InceptionResnet的组织形式  不同模型需要修改不同名称
    train_dataset=MyDataSet(json_Description=ANNOTATION_TRAIN,transform=preprocess_with_augmentation(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_TRAIN_PRE)
    val_dataset=MyDataSet(json_Description=ANNOTATION_VAL,transform=preprocess(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_VAL_PRE)
    train_dataLoader=DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,num_workers=16,shuffle=True)
    val_dataLoader=DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,num_workers=1,shuffle=False)
    model=getmodel()
    weight=torch.Tensor([1,3,3,3,3,4,2,3,3,3,3,3,3,3,3,3,2,3,3,3,2,3,4,2,3,1,1,3,2,2,1,3,3,1,3,2,3,3,3,3,2,1,3,2,3,3,3,1,3,3,4,4,3,2,2,3,1,1,3]).cuda()
    criterion=nn.CrossEntropyLoss(weight=weight).cuda()
#     lx, px = utils.predict(model,val_dataLoader)
#     min_loss = criterion(Variable(px), Variable(lx)).item()
    min_loss=4.1
    print('min_loss is :%f'%(min_loss))
    min_acc=0.80
    patience=0
    lr=0.0
    momentum=0.0
    for epoch in range(epochNum):
        print('Epoch {}/{}'.format(epoch, epochNum - 1))
        print('-' * 10)
        if epoch==3:
            lr=1e-4
            momentum=0.9
            print('set lr=:%f,momentum=%f'%(lr,momentum))
        if patience==2:
            patience=0
            model.load_state_dict(torch.load('../model/ResNet/'+date+'_loss_best.pth')['state_dict'])
            lr=lr/10
            print('loss has increased lr divide 10 lr now is :%f'%(lr))
        if epoch==0 or epoch==1 or epoch==2: #第一轮首先训练全连接层
            lr=1e-3
#             optimizer=torch.optim.SGD(params=model.fresh_params(),lr=lr,momentum=0.9)
            optimizer = torch.optim.Adam(model.fresh_params(),lr = lr,amsgrad=True,weight_decay=1e-4)
        else:
            optimizer = torch.optim.Adam(model.parameters(),lr = lr,amsgrad=True,weight_decay=1e-4)
#             optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
        running_loss = RunningMean()
        running_corrects = RunningMean()
        for batch_idx, (inputs, labels) in enumerate(train_dataLoader):
            model.train(True)
            n_batchsize=inputs.size(0)
            inputs = Variable(inputs).cuda()
            labels = Variable(labels).cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            if isinstance(outputs,tuple):
                loss=sum((criterion(o,labels)) for o in outputs)
            else:
                loss = criterion(outputs, labels)
            running_loss.update(loss.item(),1)
            running_corrects.update(torch.sum(preds == labels.data).data,n_batchsize)
            loss.backward()
            optimizer.step()
            if batch_idx%30==29:
                print('[epoch:%d,batch:%d]:acc: %f,loss:%f'%(epoch,batch_idx,running_corrects.value,running_loss.value))
                if batch_idx%300==299: 
                    niter = epoch * len(train_dataset)/BATCH_SIZE + batch_idx
                    writer.add_scalar('Train/Acc',running_corrects.value,niter)
                    writer.add_scalar('Train/Loss',running_loss.value,niter)
                    lx,px=utils.predict(model,val_dataLoader)
                    log_loss = criterion(Variable(px), Variable(lx))
                    log_loss = log_loss.item()
                    _, preds = torch.max(px, dim=1)
                    accuracy = torch.mean((preds == lx).float())
                    writer.add_scalar('Val/Acc',accuracy,niter)
                    writer.add_scalar('Val/Loss',log_loss,niter)
                    print('[epoch:%d,batch:%d]: val_loss:%f,val_acc:%f,val_total:%d'%(epoch,batch_idx,log_loss,accuracy,len(val_dataset)))
        print('[epoch:%d] :acc: %f,loss:%f,lr:%f,patience:%d'%(epoch,running_corrects.value,running_loss.value,lr,patience))       
        lx,px=utils.predict(model,val_dataLoader)
        log_loss = criterion(Variable(px), Variable(lx))
        log_loss = log_loss.item()
        _, preds = torch.max(px, dim=1)
        accuracy = torch.mean((preds == lx).float())
        writer.add_scalar('Val/Acc',accuracy,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        writer.add_scalar('Val/Loss',log_loss,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        print('[epoch:%d]: val_loss:%f,val_acc:%f,'%(epoch,log_loss,accuracy))
        if  log_loss < min_loss:
            utils.snapshot('../model/', 'ResNet', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy })          
            patience = 0
            min_loss=log_loss
            print('save new model loss,now loss is ',min_loss)
        else:
            patience += 1
        if accuracy>min_acc:
            utils.snapshot('../model/', 'ResNet', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy },key='acc') 
            min_acc=accuracy
            print('save new model acc,now acc is ',min_acc)

In [6]:
def TrainWithRawData(path,epochNum):
    train_dataset=MyDataSet(json_Description=ANNOTATION_TRAIN,transform=preprocess(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_TRAIN_PRE)
    val_dataset=MyDataSet(json_Description=ANNOTATION_VAL,transform=preprocess(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_VAL_PRE)
    train_dataLoader=DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,num_workers=16,shuffle=True)
    val_dataLoader=DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,num_workers=1,shuffle=False)
    model=getmodel()
    criterion=nn.CrossEntropyLoss().cuda()
    modelParams=torch.load(path)
    model.load_state_dict(modelParams['state_dict'])
    min_loss=modelParams['val_loss']
    print('min_loss is :%f'%(min_loss))
    print('val_correct is %f'%(modelParams['val_correct']))
    min_acc=max(modelParams['val_correct'],0.81)
    optinizerSave=modelParams['optimizer']
    patience=0
    lr=1e-4
    momentum=0.9
    beginepoch=modelParams['epoch']
    for epoch in range(beginepoch,epochNum):
        print('Epoch {}/{}'.format(epoch, epochNum - 1))
        print('-' * 10)
        if patience==3:
            patience=0
            model.load_state_dict(torch.load('../model/ResNet/'+date+'_loss_best.pth')['state_dict'])
            lr=lr/10
            print('lr desencd')
        if epoch==beginepoch:
            optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
#             optimizer.load_state_dict(optinizerSave)
#             lr=optimizer['lr']
#             momentum=optimizer['momentum']
            print('begin lr is ',lr)
            
        else:
            optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
                   
        running_loss = RunningMean()
        running_corrects = RunningMean()
        for batch_idx, (inputs, labels) in enumerate(train_dataLoader):
            model.train(True)
            n_batchsize=inputs.size(0)
            inputs = Variable(inputs).cuda()
            labels = Variable(labels).cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            if isinstance(outputs,tuple):
                loss=sum((criterion(o,labels)) for o in outputs)
            else:
                loss = criterion(outputs, labels)
            running_loss.update(loss.item(),1)
            running_corrects.update(torch.sum(preds == labels.data).data,n_batchsize)
            loss.backward()
            optimizer.step()
            if batch_idx%30==29:
                print('[epoch:%d,batch:%d]:acc: %f,loss:%f'%(epoch,batch_idx,running_corrects.value,running_loss.value))
                if batch_idx%300==299: 
                    niter = epoch * len(train_dataset)/BATCH_SIZE + batch_idx
                    lx,px=utils.predict(model,val_dataLoader)
                    log_loss = criterion(Variable(px), Variable(lx))
                    log_loss = log_loss.item()
                    _, preds = torch.max(px, dim=1)
                    accuracy = torch.mean((preds == lx).float())
                    print('[epoch:%d,batch:%d]: val_loss:%f,val_acc:%f,val_total:%d'%(epoch,batch_idx,log_loss,accuracy,len(val_dataset)))
                    if  log_loss < min_loss:
                        utils.snapshot('../model/', 'ResNet', {
                               'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optimizer': optimizer.state_dict(),
                               'val_loss': log_loss,
                               'val_correct':accuracy })          

                        min_loss=log_loss
                        print('save new model loss,now loss is ',min_loss)

                    if accuracy>min_acc:
                        utils.snapshot('../model/', 'ResNet', {
                               'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optimizer': optimizer.state_dict(),
                               'val_loss': log_loss,
                               'val_correct':accuracy },key='acc') 
                        min_acc=accuracy
                        print('save new model acc,now acc is ',min_acc)
        print('[epoch:%d] :acc: %f,loss:%f,lr:%f,patience:%d'%(epoch,running_corrects.value,running_loss.value,lr,patience))         
        lx,px=utils.predict(model,val_dataLoader)
        log_loss = criterion(Variable(px), Variable(lx))
        log_loss = log_loss.item()
        _, preds = torch.max(px, dim=1)
        accuracy = torch.mean((preds == lx).float())
        print('[epoch:%d]: val_loss:%f,val_acc:%f,'%(epoch,log_loss,accuracy))
        if  log_loss < min_loss:
            utils.snapshot('../model/', 'ResNet', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy })          
            patience = 0
            min_loss=log_loss
            print('save new model loss,now loss is ',min_loss)
        else:
            patience += 1
        if accuracy>min_acc:
            utils.snapshot('../model/', 'ResNet', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy },key='acc') 
            min_acc=accuracy
            print('save new model acc,now acc is ',min_acc)

In [ ]:
TrainWithRawData('../model/ResNet/2018-10-30_acc_best.pth',60)

[+] loading model... Done
min_loss is :0.375886
val_correct is 0.858779
Epoch 15/59
----------
begin lr is  0.0001
[epoch:15,batch:29]:acc: 0.871875,loss:0.289995
[epoch:15,batch:59]:acc: 0.877604,loss:0.279860
[epoch:15,batch:89]:acc: 0.878819,loss:0.283847
[epoch:15,batch:119]:acc: 0.884635,loss:0.274498
[epoch:15,batch:149]:acc: 0.882708,loss:0.278735
[epoch:15,batch:179]:acc: 0.885417,loss:0.277195
[epoch:15,batch:209]:acc: 0.883185,loss:0.279258
[epoch:15,batch:239]:acc: 0.882422,loss:0.278958
[epoch:15,batch:269]:acc: 0.882292,loss:0.277777
[epoch:15,batch:299]:acc: 0.881146,loss:0.278261
[epoch:15,batch:299]: val_loss:0.349366,val_acc:0.862304,val_total:4539
save new model loss,now loss is  0.3493664860725403
save new model acc,now acc is  tensor(0.8623, device='cuda:0')
[epoch:15,batch:329]:acc: 0.881345,loss:0.277557
[epoch:15,batch:359]:acc: 0.881250,loss:0.276392
[epoch:15,batch:389]:acc: 0.880128,loss:0.277674
[epoch:15,batch:419]:acc: 0.879167,loss:0.278443
[epoch:15,batch

[epoch:18,batch:869]:acc: 0.896659,loss:0.245269
[epoch:18,batch:899]:acc: 0.897014,loss:0.244852
[epoch:18,batch:899]: val_loss:0.338935,val_acc:0.868033,val_total:4539
save new model acc,now acc is  tensor(0.8680, device='cuda:0')
[epoch:18,batch:929]:acc: 0.896438,loss:0.245278
[epoch:18,batch:959]:acc: 0.896517,loss:0.245471
[epoch:18,batch:989]:acc: 0.896402,loss:0.245895
[epoch:18] :acc: 0.896333,loss:0.246377,lr:0.000100,patience:1
[epoch:18]: val_loss:0.338687,val_acc:0.867592,
Epoch 19/59
----------
[epoch:19,batch:29]:acc: 0.898958,loss:0.237218
[epoch:19,batch:59]:acc: 0.893750,loss:0.248203
[epoch:19,batch:89]:acc: 0.890625,loss:0.250446
[epoch:19,batch:119]:acc: 0.894271,loss:0.247779
[epoch:19,batch:149]:acc: 0.893542,loss:0.251480
[epoch:19,batch:179]:acc: 0.895139,loss:0.245447
[epoch:19,batch:209]:acc: 0.894345,loss:0.245270
[epoch:19,batch:239]:acc: 0.894401,loss:0.245173
[epoch:19,batch:269]:acc: 0.893981,loss:0.246789
[epoch:19,batch:299]:acc: 0.894583,loss:0.246021

[epoch:22,batch:809]:acc: 0.904128,loss:0.227238
[epoch:22,batch:839]:acc: 0.904129,loss:0.227655
[epoch:22,batch:869]:acc: 0.904526,loss:0.227169
[epoch:22,batch:899]:acc: 0.904306,loss:0.227561
[epoch:22,batch:899]: val_loss:0.334947,val_acc:0.869575,val_total:4539
save new model loss,now loss is  0.33494654297828674
[epoch:22,batch:929]:acc: 0.904301,loss:0.227937
[epoch:22,batch:959]:acc: 0.904004,loss:0.228362
[epoch:22,batch:989]:acc: 0.903788,loss:0.228709
[epoch:22] :acc: 0.903806,loss:0.228772,lr:0.000010,patience:2
[epoch:22]: val_loss:0.336432,val_acc:0.868473,
Epoch 23/59
----------
lr desencd
[epoch:23,batch:29]:acc: 0.892708,loss:0.244052
[epoch:23,batch:59]:acc: 0.892708,loss:0.246488
[epoch:23,batch:89]:acc: 0.897917,loss:0.239494
[epoch:23,batch:119]:acc: 0.904687,loss:0.234706
[epoch:23,batch:149]:acc: 0.904583,loss:0.234311
[epoch:23,batch:179]:acc: 0.906076,loss:0.232205
[epoch:23,batch:209]:acc: 0.905655,loss:0.228710
[epoch:23,batch:239]:acc: 0.904687,loss:0.22840

[epoch:26,batch:899]:acc: 0.903993,loss:0.227690
[epoch:26,batch:899]: val_loss:0.335389,val_acc:0.869354,val_total:4539
[epoch:26,batch:929]:acc: 0.903831,loss:0.228141
[epoch:26,batch:959]:acc: 0.903581,loss:0.228291
[epoch:26,batch:989]:acc: 0.904009,loss:0.227532
[epoch:26] :acc: 0.903995,loss:0.227785,lr:0.000000,patience:0
[epoch:26]: val_loss:0.341298,val_acc:0.868694,
Epoch 27/59
----------
[epoch:27,batch:29]:acc: 0.908333,loss:0.224235
[epoch:27,batch:59]:acc: 0.901042,loss:0.232707
[epoch:27,batch:89]:acc: 0.901389,loss:0.230769
[epoch:27,batch:119]:acc: 0.900521,loss:0.232782
[epoch:27,batch:149]:acc: 0.898542,loss:0.234477
[epoch:27,batch:179]:acc: 0.897569,loss:0.235424
[epoch:27,batch:209]:acc: 0.901042,loss:0.230325
[epoch:27,batch:239]:acc: 0.901563,loss:0.230396
[epoch:27,batch:269]:acc: 0.900463,loss:0.231558
[epoch:27,batch:299]:acc: 0.900937,loss:0.232109
[epoch:27,batch:299]: val_loss:0.335757,val_acc:0.867592,val_total:4539
[epoch:27,batch:329]:acc: 0.900663,loss